Here we produce min/max/average forest maps on 100m grid

First lets check the extent/resolution/nodata values of all our input files
a list of input files is stored in filelist_stack100m_vrt.txt

We need to add the "CLC has data" mask

In [3]:
!ls out_data/GLADGLCextent

reproj3035_100m_tifs  tilelist.txt


In [40]:
!cat out_data/filelist_stack100m_vrt.txt

out_data/C-GLOPS/2019/Tree_CoverFraction_2019_mean100m_0nodata.tif
out_data/CLC/CLC_recode01.tif
out_data/CLC10m/mean100m_CLC10.tif
out_data/EPFD/Delta_3035.tif
out_data/IFL/IFL2020_3035.tif


In [1]:
%%bash
cat out_data/filelist_stack100m_vrt.txt | while read line 
do
basename $line
gdalinfo $line | grep NoData
done

Tree_CoverFraction_2019_mean100m_0nodata.tif
  NoData Value=0
CLC_recode01.tif
  NoData Value=48
mean100m_CLC10.tif
  NoData Value=48
Delta_3035.tif
  NoData Value=0
IFL2020_3035.tif
  NoData Value=0


IFL is just patches, so we want to ignore 0 values (therefore they are coded as NA)
Similar for EPFD/Delta_3035

TODO: this shouldn't have nodata values only zeros out_data/C-GLOPS/2019/Tree_CoverFraction_2019_mean100m_0nodata.tif 

In [100]:
%%bash
cat out_data/filelist_stack100m_vrt.txt | while read line 
do
basename $line
gdalinfo -mm $line | grep Computed
done



Tree_CoverFraction_2019_mean100m_0nodata.tif
    Computed Min/Max=0.010,1.000
CLC_recode01.tif
    Computed Min/Max=0.000,1.000
mean100m_CLC10.tif
    Computed Min/Max=0.000,1.000
Delta_3035.tif
    Computed Min/Max=1.000,1.000
IFL2020_3035.tif
    Computed Min/Max=1.000,1.000


In [66]:
%%bash
cat out_data/filelist_stack100m_vrt.txt | while read line 
do
basename $line
paste  <(gdalinfo $line | grep NoData) <(gdalinfo $line | awk '/Type/  {type=substr($4, 6, 7)}
/Pixel Size/  {pxsize=$0} /Origin/  {ori=$0} END {print type " " pxsize " " ori}')
done


Tree_CoverFraction_2019_mean100m_0nodata.tif
  NoData Value=0	Float64 Pixel Size = (100.000000000000000,-100.000000000000000) Origin = (900000.000000000000000,5500000.000000000000000)
CLC_recode01.tif
  NoData Value=48	Byte, Pixel Size = (100.000000000000000,-100.000000000000000) Origin = (900000.000000000000000,5500000.000000000000000)
mean100m_CLC10.tif
  NoData Value=48	Float64 Pixel Size = (100.000000000000000,-100.000000000000000) Origin = (900000.000000000000000,5500000.000000000000000)
Delta_3035.tif
  NoData Value=0	Byte, Pixel Size = (100.000000000000000,-100.000000000000000) Origin = (900000.000000000000000,5500000.000000000000000)
IFL2020_3035.tif
  NoData Value=0	Byte, Pixel Size = (100.000000000000000,-100.000000000000000) Origin = (900000.000000000000000,5500000.000000000000000)


In [8]:
%%bash
start=`date +%s`

outfile=out_data/hybrid/forest_hybrid_mean.tif

pkcomposite $(for file in $(cat out_data/filelist_stack100m_vrt.txt  ); do echo -i $file; done) \
-o $outfile \
-m out_data/CLCreference_ocean.tif -msknodata 0 \
-srcnodata 48 \
-cr mean -dstnodata 0 \
-co  COMPRESS=LZW -co ZLEVEL=9 



end=`date +%s`

expr $end - $start

Process is interrupted.


TODO find a spot where the IFL is 1 and see if it got handled correctly. 
Look at coastlines 